In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

deep_learning = tf.constant('Deep Learning')
session = tf.Session()
session.run(deep_learning)



Instructions for updating:
non-resource variables are not supported in the long term


b'Deep Learning'

In [2]:
a = tf.constant(2)
b = tf.constant(3)
multiply = tf.multiply(a,b)

session.run(multiply)

6

In [3]:
weighs = tf.Variable(tf.random_normal([300,200], stddev=0.5), name = 'weights')

In [4]:
weighs = tf.Variable(tf.random_normal([300,200], stddev=0.5), name = 'weights', trainable=False)

In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

shape = [2,3]

tf.zeros(shape, dtype=tf.float32, name=None)
tf.ones(shape, dtype=tf.float32, name=None)
tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
tf.random_uniform(shape, minval=0.0, maxval=None, dtype=tf.float32, seed=None, name=None)

<tf.Tensor 'random_uniform:0' shape=(2, 3) dtype=float32>

In [13]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x = tf.placeholder(tf.float32, name='x', shape=[None, 784])
W = tf.Variable(tf.random_uniform([784, 10], -1, 1), name='W')
b = tf.Variable(tf.zeros([10]), name = 'biases')

output = tf.matmul(x, W) + b

init_op = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init_op)
feed_dict = {'x': get_minibatch()}
sess.run(output, feed_dict=feed_dict)

NameError: name 'get_minibatch' is not defined

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce MX350, pci bus id: 0000:2d:00.0, compute capability: 6.1



In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0], shape=[2,2], name='a')
    b = tf.constant([1.0, 2.0], shape=[2,1], name='b')
    c = tf.matmul(a,b)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(c)

Instructions for updating:
non-resource variables are not supported in the long term
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce MX350, pci bus id: 0000:2d:00.0, compute capability: 6.1



array([[ 5.],
       [11.]], dtype=float32)

In [1]:
import tensorflow as tf 
tf.__version__

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11267140416806079922,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1393599694
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5676611673115933704
 physical_device_desc: "device: 0, name: GeForce MX350, pci bus id: 0000:2d:00.0, compute capability: 6.1"]

In [13]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

learning_rate = 0.01
training_epochs = 1000
batch_size = 100
display_step = 1
init = tf.constant_initializer(1)

def inference(x):
    tf.constant_initializer(value=0)
    W = tf.get_variable('W', [784, 10], initializer=init)
    b = tf.get_variable('b', [10], initializer=init)
    output = tf.nn.softmax(tf.matmul(x,W)+b)
    return output

def loss(output, y):
    dot_product = y*tf.log(output)
    xentropy = -tf.reduce_sum(dot_product, reduction_indices=1)
    
    loss = tf.reduce_mean(xentropy)
    
    return loss

def training(cost, global_step):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

def evaluate(outpuy, y):
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

def training(cost, global_step):
    tf.summary.scalar('cost', cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

with tf.Graph().as_default():
    x = tf.placeholder('float', [None, 784])
    y = tf.placeholder('float', [None, 10])
    
    output = inference(x)
    cost = loss(output,y)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    
    trian_op = training(cost, global_step)
    eval_op = evaluate(output, y)
    summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    sess = tf.Session()
    summary_writer = tf.summary.FileWriter('logistic_logs/', graph_def=sess.graph_def)
    init_op = tf.initialize_all_variables()
    
    sess.run(init_op)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            mbatch_x, mbatch_y = mnist.train.next_batch(batch_size)
            
            feed_dict = {x : mbatch_x, y : mbatch_y}
            sess.run(train_op, feed_dict=feed_dict)
            
            minibatch_cost = sess.run(cost, feed_dict=feed_dict)
            avg_cost += minibatch_cost/total_batch
            
        if epoch % display_step == 0:
            val_feed_dict = { x: mnist.validation.images, y: mnist.validation.labels}
            accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
            
            print('Validaiotn Error:', (1-accuracy))
            
            summary_str = sess.run(summary_op, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, sess.run(global_step))
            
            saver.save(sess, 'logisitic_logs/model-checkpoint', global_step=global_step)
    print('Optimization Finished!')
    
    test_feed_dict = { x : mnist.test.images, y: mnist.test.labels}
    accuracy = sess.run(eval_op, feed_dict=test_feed_dict)
    
    print('Test Accuracy:'. accuracy)

ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'